In [3]:
import PySimpleGUI as sg

sg.theme('BluePurple')
sg.Print('Re-routing the stdout', do_not_reroute_stdout=False)
layout = [[sg.Text('Progress:'),sg.Text(size=(15,1), key='-OUTPUT-')],
          [sg.Button('Import Dataset and packages')],
          [sg.Button('Analyse the dataset')],
          [sg.Button('Show Imbalance in the dataset')],
          [sg.Button('Preprocess dataset')],
          [sg.Button('Perform sampling'),sg.Button('Show sampling results')],
          [sg.Button('Split dataset and train algorithms')],
          [sg.Button('Generate confusion matrix')],
          [sg.Button('Algorithm evaluation')],
          [sg.Button('Give user input')],
          [sg.Button('Send E-Mail')],
          [sg.Text('Prediction:'), sg.Text(size=(15,1), key='prediction')],
          [sg.Button('Exit')]]
 
window = sg.Window('Credit card Fraud Detection', layout)
event, values = window.read()

while True:  # Event Loop
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Exit':
        break
        
    if event == 'Import Dataset and packages':
       
        import numpy as np 
        import pandas as pd 
        import matplotlib.pyplot as plt 
        import seaborn as sns 
        from matplotlib import gridspec
        import random
        import math
        window['-OUTPUT-'].update("Importing...")
        
        
        path ="creditcard.csv"
        data = pd.read_csv(path)
        data.head()
        
        print(data.shape) 
        print(data.describe())
        window['-OUTPUT-'].update("Dataset done")
        
        
    if event == 'Analyse the dataset':
        fraud = data[data['Class'] == 1] 
        valid = data[data['Class'] == 0] 
        outlierFraction = len(fraud)/float(len(valid)) 
        print(outlierFraction) 
        fraudcases=len(data[data['Class'] == 1])
        normalcases=len(data[data['Class'] == 0])
        print('Fraud Cases: {}'.format(len(data[data['Class'] == 1]))) 
        print('Valid Transactions: {}'.format(len(data[data['Class'] == 0])))
        window['-OUTPUT-'].update("Analysis done")
        layout1 = [[sg.Text('Results of analysis')],
                   [sg.Text('Fraud cases:'),sg.Text(size=(15,1), key='-OUTPUT1-')],
                   [sg.Text('Normal Cases:'),sg.Text(size=(15,1), key='-OUTPUT2-')],
                   [sg.Button("Show"),sg.Cancel()]]
        window1 = sg.Window('Data Analysis', layout1)
        while True:
            event1, values1 = window1.read()
            if event1 == "Show":
                window1['-OUTPUT1-'].update(fraudcases)
                window1['-OUTPUT2-'].update(normalcases)
            if event1 == 'Cancel':
                break
        window1.close()
        
        
    if event == 'Show Imbalance in the dataset':
        from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
        plt.figure(figsize=(7,5))
        sns.countplot(data['Class'])
        plt.title("Class Count", fontsize=18)
        plt.xlabel("Is fraud?", fontsize=15)
        plt.ylabel("Count", fontsize=15)


        def draw_figure(canvas, figure, loc=(0, 0)):
            figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
            figure_canvas_agg.draw()
            figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=1)
            return figure_canvas_agg

        sg.theme('Light Brown 3')

        fig = plt.gcf()  # if using Pyplot then get the figure from the plot
        figure_x, figure_y, figure_w, figure_h = fig.bbox.bounds

        # define the window layout
        layout2 = [[sg.Text('Bar graph', font='Any 18')],
          [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS1-')],
          [sg.OK(pad=((figure_w / 2, 0), 3), size=(4, 2))]]

        # create the form and show it without the plot
        window2 = sg.Window('Bar Graph showing Imbalance',layout2, force_toplevel=True, finalize=True)

        # add the plot to the window
        fig_photo = draw_figure(window2['-CANVAS1-'].TKCanvas, fig)

        # show it all again and get buttons
        event2, values2 = window2.read()

        window2.close()
        plt.clf()
        
    if event == 'Preprocess dataset':
        #Studing the attributes of amount class 
        fraud.Amount.describe() 
        valid.Amount.describe()
        window['-OUTPUT-'].update('Processing, please wait..')
        
        #Time feature engineering
        timedelta = pd.to_timedelta(data['Time'], unit='s')
        data['Time_min'] = (timedelta.dt.components.minutes).astype(int)
        data['Time_hour'] = (timedelta.dt.components.hours).astype(int)
        
        #Graphs showing relationship between time and output class
        #plt.figure(figsize=(12,5))
        #sns.distplot(data[data['Class'] == 0]["Time_hour"],color='g')
        #sns.distplot(data[data['Class'] == 1]["Time_hour"],color='r')
        #plt.title('Fraud x Normal Transactions by Hours', fontsize=17)
        #plt.xlim([-1,25])
        #plt.show()
        #plt.figure(figsize=(12,5))
        #sns.distplot(data[data['Class'] == 0]["Time_min"], color='g')
        #sns.distplot(data[data['Class'] == 1]["Time_min"], color='r')
        #plt.title('Fraud x Normal Transactions by Hours', fontsize=17)
        #plt.xlim([-1,61])
        #plt.show()
        
        data['Amount_log'] = np.log(data.Amount + 0.01)
        #plt.figure(figsize=(14,6))
        #plt.subplot(121)
        #ax = sns.boxplot(x ="Class",y="Amount",data=data)
        #ax.set_title("Class x Amount", fontsize=20)
        #ax.set_xlabel("Is Fraud?", fontsize=16)
        #ax.set_ylabel("Amount(US)", fontsize = 16)

        #plt.subplot(122)
        #ax1 = sns.boxplot(x ="Class",y="Amount_log", data=data)
        #ax1.set_title("Class x Amount", fontsize=20)
        #ax1.set_xlabel("Is Fraud?", fontsize=16)
        #ax1.set_ylabel("Amount(Log)", fontsize = 16)

        #plt.subplots_adjust(hspace = 0.6, top = 0.8)
        #plt.show()
        #ax = sns.lmplot(y="Amount", x="Time_min", fit_reg=False,aspect=1.8, data=data, hue='Class')
        #plt.title("Amounts by Minutes of Frauds and Normal Transactions",fontsize=16)
        #plt.show()
        
        #columns = data.iloc[:,1:29].columns
        #frauds = data.Class == 1
        #normals = data.Class == 0

        #grid = gridspec.GridSpec(14, 2)
        #plt.figure(figsize=(15,20*4))

        #for n, col in enumerate(data[columns]):
        #    ax = plt.subplot(grid[n])
        #    sns.distplot(data[col][frauds], bins = 50, color='g') #Will receive the "semi-salmon" violin
        #    sns.distplot(data[col][normals], bins = 50, color='r') #Will receive the "ocean" color
        #    ax.set_ylabel('Density')
        #    ax.set_title(str(col))
        #    ax.set_xlabel('')
        #plt.show()
        
        window['-OUTPUT-'].update('Preprocessing done')
        data = data[["Time_hour","Time_min","V2","V3","V4","V9","V10","V11","V12","V14","V16","V17","V18","V19","V27","Amount","Class"]]
        data.Amount = np.log(data.Amount + 0.001)
        
        
    if event == 'Perform sampling':
        X = data.drop(["Class"], axis=1).values #Setting the X to do the split
        y = data["Class"].values # transforming the values in array
        #from imblearn.pipeline import make_pipeline as make_pipeline_imb
        from imblearn.over_sampling import SMOTE
        smote=SMOTE(random_state=42)
        X_smote,y_smote=smote.fit_resample(X,y)
        window['-OUTPUT-'].update("Sampling done.")
        
        
    if event == 'Show sampling results':
        from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
        plt.figure(figsize=(7,5))
        sns.countplot(y_smote)
        plt.title("Class Count", fontsize=18)
        plt.xlabel("Is fraud?", fontsize=15)
        plt.ylabel("Count", fontsize=15)


        def draw_figure(canvas, figure, loc=(0, 0)):
            figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
            figure_canvas_agg.draw()
            figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=1)
            return figure_canvas_agg

        sg.theme('Light Brown 3')

        fig = plt.gcf()  # if using Pyplot then get the figure from the plot
        figure_x, figure_y, figure_w, figure_h = fig.bbox.bounds

        # define the window layout
        layout3 = [[sg.Text('Sampling results', font='Any 18')],
          [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS3-')],
          [sg.OK(pad=((figure_w / 2, 0), 3), size=(4, 2))]]

        # create the form and show it without the plot
        window3 = sg.Window('SAMPLING RESULTS',layout3, force_toplevel=True, finalize=True)

        # add the plot to the window
        fig_photo = draw_figure(window3['-CANVAS3-'].TKCanvas, fig)

        # show it all again and get buttons
        event3, values3 = window3.read()

        window3.close()
        plt.clf()
    if event == 'Split dataset and train algorithms':
        from sklearn.model_selection import train_test_split
        from collections import Counter
        from imblearn.pipeline import make_pipeline as make_pipeline_imb # To do our transformation in a unique time
        from imblearn.over_sampling import SMOTE
        from sklearn.pipeline import make_pipeline
        from imblearn.metrics import classification_report_imbalanced
        X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, random_state=2, test_size=0.20)
        from sklearn.metrics import precision_score, recall_score, fbeta_score, confusion_matrix, precision_recall_curve, accuracy_score
        def print_results(headline, true_value, pred):
            print(headline)
            print("accuracy: {}".format(accuracy_score(true_value, pred)))
            print("precision: {}".format(precision_score(true_value, pred)))
            print("recall: {}".format(recall_score(true_value, pred)))
            print("f2: {}".format(fbeta_score(true_value, pred, beta=2)))
        partialX_train=X_train[:10000,:]
        partialy_train=y_train[:10000]
        print(partialX_train.shape)
        print(partialy_train.shape)        
        from sklearn.ensemble import RandomForestClassifier 
        # random forest model creation 
        rfc = RandomForestClassifier(random_state=42) 
        rfc.fit(partialX_train, partialy_train) 
        # predictions 
        yPred = rfc.predict(X_test)                                                                                                     
        print_results("\nSMOTE + RandomForest classification", y_test, yPred)
        from sklearn.metrics import classification_report, accuracy_score  
        from sklearn.metrics import precision_score, recall_score 
        from sklearn.metrics import f1_score, matthews_corrcoef 
        from sklearn.metrics import confusion_matrix 

        n_errors = (yPred != y_test).sum() 
        print("The model used is Random Forest classifier") 
      
        acc = accuracy_score(y_test, yPred) 
        print("The accuracy is {}".format(acc)) 
  
        prec = precision_score(y_test, yPred) 
        print("The precision is {}".format(prec)) 
  
        rec = recall_score(y_test, yPred) 
        print("The recall is {}".format(rec)) 
  
        f1 = f1_score(y_test, yPred) 
        print("The F1-Score is {}".format(f1)) 
  
        MCC = matthews_corrcoef(y_test, yPred) 
        print("The Matthews correlation coefficient is{}".format(MCC))
        
        from sklearn import svm
        clf_SVM = svm.SVC(C=1,kernel="linear")
        clf_SVM.fit(partialX_train,partialy_train)
        y_pred_svm  = clf_SVM.predict(X_test)
        print_results("\nSMOTE + SVM classification", y_test, y_pred_svm)
        window['-OUTPUT-'].update('Split & training done.')
        
        
        
    if event == 'Generate confusion matrix':
        from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
        LABELS = ['Normal', 'Fraud'] 
        conf_matrix = confusion_matrix(y_test, yPred) 
        print(conf_matrix)
        plt.figure(figsize =(8,8)) 
        sns.heatmap(conf_matrix, xticklabels = LABELS, yticklabels = LABELS, annot = True, fmt ="d"); 
        plt.title("Confusion matrix") 
        plt.ylabel('True class') 
        plt.xlabel('Predicted class') 


        def draw_figure(canvas, figure, loc=(0, 0)):
            figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
            figure_canvas_agg.draw()
            figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=1)
            return figure_canvas_agg

        sg.theme('Light Brown 3')

        fig = plt.gcf()  # if using Pyplot then get the figure from the plot
        figure_x, figure_y, figure_w, figure_h = fig.bbox.bounds

        # define the window layout
        layout4 = [[sg.Text('Confusion matrix', font='Any 18')],
          [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS4-')],
          [sg.OK(pad=((figure_w / 2, 0), 3), size=(4, 2))]]

        # create the form and show it without the plot
        window4 = sg.Window('Confusion matrix',layout4, force_toplevel=True, finalize=True)

        # add the plot to the window
        fig_photo = draw_figure(window4['-CANVAS4-'].TKCanvas, fig)

        # show it all again and get buttons
        event4, values4 = window4.read()

        window4.close()
        plt.clf()
        
    if event == 'Algorithm evaluation':
        RFacc =0.9853507553242002
        SVMacc =0.9382287251815767
        
        print("RF:{},\nSVM:{} ".format(RFacc,SVMacc))
        from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
        accuracies=[RFacc,SVMacc]
        algorithms=["Random Forest","Support Vector Machine"]
        acc_data = pd.DataFrame({'Algorithms': algorithms, 'Accuracies': accuracies})
        acc_data = acc_data.sort_values(by='Accuracies',ascending=False)
        g = sns.barplot(x='Algorithms',y='Accuracies',data=acc_data)
        g.set_xticklabels(g.get_xticklabels(),rotation=90)
        g.set_title('Performance evaluation of Algorithms',fontsize=20)


        def draw_figure(canvas, figure, loc=(0, 0)):
            figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
            figure_canvas_agg.draw()
            figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=1)
            return figure_canvas_agg

        sg.theme('Light Brown 3')

        fig = plt.gcf()  # if using Pyplot then get the figure from the plot
        figure_x, figure_y, figure_w, figure_h = fig.bbox.bounds

        # define the window layout
        layout5 = [[sg.Text('Algorithm Evaluation', font='Any 18')],
          [sg.Canvas(size=(figure_w, figure_h), key='-CANVAS5-')],
          [sg.OK(pad=((figure_w / 2, 0), 3), size=(4, 2))]]

        # create the form and show it without the plot
        window5 = sg.Window('Algorithm evaluation',layout5, force_toplevel=True, finalize=True)

        # add the plot to the window
        fig_photo = draw_figure(window5['-CANVAS5-'].TKCanvas, fig)

        # show it all again and get buttons
        event5, values5 = window5.read()

        window5.close()
        plt.clf()
        window['-OUTPUT-'].update('Evalution done')
    if event == "Give user input":
        import PySimpleGUI as sg
        from datetime import datetime

        sg.theme('Topanga')      # Add some color to the window

        layout6 = [
            [sg.Text('Please enter your Credit card Details')],
            [sg.Text('Card holder name', size=(15, 1)), sg.InputText()],
            [sg.Text('Card number', size=(15, 1)), sg.InputText()],
            [sg.Text('Item name', size=(15, 1)), sg.InputText()],
            [sg.Text('Amount', size=(15, 1)), sg.InputText()],
            [sg.Submit(), sg.Cancel()]
            
        ]

        window6 = sg.Window('Credit card transaction window', layout6)
        event6, values6 = window6.read()
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        time_values=current_time.split(":")
        time_hour=time_values[0]
        time_min=time_values[1]
        name=values6[0]
        card_number=values6[1]
        item_name=values6[2]
        amount=values6[3]
        window6.close()

        
        #input validation
        try:
            card_number = float(card_number)
            amount = float(amount)
            if (not name.isalpha()): 
                raise ValueError 
            
        except ValueError:
            window['prediction'].update("ERROR")
            continue
            
        print("1")
        choices=list()
        for i in range(len(partialy_train)):
            if(partialy_train[i]==1):
                choices.append(partialX_train[i][2:-1])
            if(len(choices)==20):
                break
        print("2")
        for i in range(len(partialy_train)):
            if(partialy_train[i]==0):
                choices.append(partialX_train[i][2:-1])
                if(len(choices)==40):
                    break
        print("3")
        import random
        n = random.randint(0,40)
        print(n)
        v1=choices[n][0]
        v2=choices[n][1]
        v3=choices[n][2]
        v4=choices[n][3]
        v5=choices[n][4]
        v6=choices[n][5]
        v7=choices[n][6]
        v8=choices[n][7]
        v9=choices[n][8]
        v10=choices[n][9]
        v11=choices[n][10]
        v12=choices[n][11]
        v13=choices[n][12]
        out = rfc.predict([[float(time_hour),float(time_min),v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,float(math.log(amount+.001))]])
        print(out)
        result=0
        if out[0]==1:
            result="FRAUD!!!"
        else:
            result="Safe transaction"
        window['prediction'].update(result)
    
    
    if event == "Send E-Mail":
        import smtplib,ssl
        msg = "A transaction has taken place with your credit card number "+str(card_number)+" The transaction has been flagged as "+result+" transaction by our system. Please take necessary action if necesarry."
  
        smtp = smtplib.SMTP('smtp.gmail.com', 587  )
        smtp.starttls()
        smtp.login('karthik.raghoji.24@gmail.com',"34064568142345")
        smtp.sendmail('karthik.raghoji.24@gmail.com', 'amoghlp699@gmail.com', msg)
        smtp.quit()
        window['-OUTPUT-'].update('Email Sent')
        
        
window.close()

<Figure size 432x288 with 0 Axes>